<a href="https://colab.research.google.com/github/PHW22/Assignment/blob/main/GenerativeAI/HW9_PHW%E3%80%8CQuick_Summary_of_Lecture_Video%E3%80%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenAI HW9: Quick Summary of Lecture Video (演講影片快速摘要)
## Objectives
- ### Learn to quickly build applications related to speech recognition using existing APIs. (學習以現成的API快速搭建語音辨識相關的應用。)


#### If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to ntu-gen-ai-2024-spring-ta@googlegroups.com

# Part1 - Preparation

## The lecture video provided for this assignment

(1) For ease of processing, it has already been converted to a MP3 file.

(2) If you would like to view the original video, the link is here:

- 李琳山教授 信號與人生 (2023)

  - https://www.youtube.com/watch?v=MxoQV4M0jY8


(3) Since the original lecture video is quite long, we have edited the segment from 1:43:24 to 2:00:49 to use for this assignment.

## Install all necessary packages and import them

The following code block takes about **150** seconds to run, but it may vary slightly depending on the condition of Colab.

In [1]:
# Install packages.
!pip install git+https://github.com/openai/whisper.git
!pip install srt
!pip install datetime
!pip install opencc
!pip install datasets
!pip install numpy
!pip install soundfile
!pip install IPython
!pip install openai
!pip install -q -U google-generativeai
!pip install anthropic

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4u16j3h1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4u16j3h1
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-m

The following code block takes about **5** seconds to run, but it may vary slightly depending on the condition of Colab.

In [2]:
# Import packages.

import whisper
import srt
import datetime
import time
import os
import re
import pathlib
import textwrap
import numpy as np
import soundfile as sf
from opencc import OpenCC
from tqdm import tqdm
from datasets import load_dataset
from openai import OpenAI
import google.generativeai as genai
import anthropic

## Download data

The code block below takes about **10** seconds to run, although there might be some slight variation depending on the state of Colab.

In [3]:
# Load dataset.
dataset_name = "kuanhuggingface/NTU-GenAI-2024-HW9"
dataset = load_dataset(dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

The code block below takes about **15** seconds to run, although there might be some slight variation depending on the state of Colab.

In [4]:
# Prepare audio.
input_audio = dataset["test"]["audio"][0]
input_audio_name = input_audio["path"]
input_audio_array = input_audio["array"].astype(np.float32)
sampling_rate = input_audio["sampling_rate"]

print(f"Now, we are going to transcribe the audio: 李琳山教授 信號與人生 (2023) ({input_audio_name}).")

Now, we are going to transcribe the audio: 李琳山教授 信號與人生 (2023) (ntu-gen-ai-2024-hw9-16k.mp3).


# Part2 - Automatic Speech Recognition (ASR)
The function "speech_recognition" aims to convert audio to subtitle.

In [5]:
def speech_recognition(model_name, input_audio, output_subtitle_path, decode_options, cache_dir="./"):
    '''
        (1) Objective:
            - This function aims to convert audio to subtitle.

        (2) Arguments:

            - model_name (str):
                The name of the model. There are five model sizes, including tiny, base, small, medium, large-v3.
                For example, you can use 'tiny', 'base', 'small', 'medium', 'large-v3' to specify the model name.
                You can see 'https://github.com/openai/whisper' for more details.

            - input_audio (Union[str, np.ndarray, torch.Tensor]):
                The path to the audio file to open, or the audio waveform
                - For example, if your input audio path is 'input.wav', you can use 'input.wav' to specify the input audio path.
                - For example, if your input audio array is 'audio_array', you can use 'audio_array' to specify the input audio array.

            - output_subtitle_path (str):
                The path of the output subtitle file.
                For example, if you want to save the subtitle file as 'output.srt', you can use 'output.srt' to specify the output subtitle path.

            - decode_options (dict):
                The options for decoding the audio file, including 'initial_prompt', 'prompt', 'prefix', 'temperature'.
                - initial_prompt (str):
                    Optional text to provide as a prompt for the first window. This can be used to provide, or
                    "prompt-engineer" a context for transcription, e.g. custom vocabularies or proper nouns
                    to make it more likely to predict those word correctly.
                    Default: None.

                You can see "https://github.com/openai/whisper/blob/main/whisper/decoding.py" and "https://github.com/openai/whisper/blob/main/whisper/transcribe.py"
                for more details.

                - temperature (float):
                    The temperature for sampling from the model. Higher values mean more randomness.
                    Default: 0.0

            - cache_dir (str):
                The path of the cache directory for saving the model.
                For example, if you want to save the cache files in 'cache' directory, you can use 'cache' to specify the cache directory.
                Default: './'

        (3) Example:

            - If you want to use the 'base' model to convert 'input.wav' to 'output.srt' and save the cache files in 'cache' directory,
            you can call this function as follows:

                speech_recognition(model_name='base', input_audio_path='input.wav', output_subtitle_path='output.srt', cache_dir='cache')
    '''

    # Record the start time.
    start_time = time.time()

    print(f"=============== Loading Whisper-{model_name} ===============")

    # Load the model.
    model = whisper.load_model(name=model_name, download_root=cache_dir)

    print(f"Begin to utilize Whisper-{model_name} to transcribe the audio.")

    # Transcribe the audio.
    transcription = model.transcribe(audio=input_audio, language=decode_options["language"], verbose=False,
                                     initial_prompt=decode_options["initial_prompt"], temperature=decode_options["temperature"])

    # Record the end time.
    end_time = time.time()

    print(f"The process of speech recognition costs {end_time - start_time} seconds.")

    subtitles = []
    # Convert the transcription to subtitle and iterate over the segments.
    for i, segment in tqdm(enumerate(transcription["segments"])):

        # Convert the start time to subtitle format.
        start_time = datetime.timedelta(seconds=segment["start"])

        # Convert the end time to subtitle format.
        end_time = datetime.timedelta(seconds=segment["end"])

        # Get the subtitle text.
        text = segment["text"]

        # Append the subtitle to the subtitle list.
        subtitles.append(srt.Subtitle(index=i, start=start_time, end=end_time, content=text))

    # Convert the subtitle list to subtitle content.
    srt_content = srt.compose(subtitles)

    print(f"\n=============== Saving the subtitle to {output_subtitle_path} ===============")

    # Save the subtitle content to the subtitle file.
    with open(output_subtitle_path, "w", encoding="utf-8") as file:
        file.write(srt_content)

In the following block, you can modify your desired parameters and the path of input file.

In [6]:
# @title Parameter Setting of Whisper { run: "auto" }

''' In this block, you can modify your desired parameters and the path of input file. '''

# The name of the model you want to use.
# For example, you can use 'tiny', 'base', 'small', 'medium', 'large-v3' to specify the model name.
# @markdown **model_name**: The name of the model you want to use.
model_name = "medium" # @param ["tiny", "base", "small", "medium", "large-v3"]

# Define the suffix of the output file.
# @markdown **suffix**: The output file name is "output-{suffix}.* ", where .* is the file extention (.txt or .srt)
suffix = "信號與人生" # @param {type: "string"}

# Path to the output file.
output_subtitle_path = f"./output-{suffix}.srt"

# Path of the output raw text file from the SRT file.
output_raw_text_path = f"./output-{suffix}.txt"

# Path to the directory where the model and dataset will be cached.
cache_dir = "./"

# The language of the lecture video.
# @markdown **language**: The language of the lecture video.
language = "zh" # @param {type:"string"}

# Optional text to provide as a prompt for the first window.
# @markdown **initial_prompt**: Optional text to provide as a prompt for the first window.
initial_prompt = "請用繁體中文" #@param {type:"string"}

# The temperature for sampling from the model. Higher values mean more randomness.
# @markdown  **temperature**: The temperature for sampling from the model. Higher values mean more randomness.
temperature = 0 # @param {type:"slider", min:0, max:1, step:0.1}

In [7]:
# Construct DecodingOptions
decode_options = {
    "language": language,
    "initial_prompt": initial_prompt,
    "temperature": temperature
}

In [8]:
# print message.
message = "Transcribe 李琳山教授 信號與人生 (2023)"
print(f"Setting: (1) Model: whisper-{model_name} (2) Language: {language} (2) Initial Prompt: {initial_prompt} (3) Temperature: {temperature}")
print(message)

Setting: (1) Model: whisper-medium (2) Language: zh (2) Initial Prompt: 請用繁體中文 (3) Temperature: 0
Transcribe 李琳山教授 信號與人生 (2023)


The code block below takes about **90 (240)** seconds to run when using the **base (medium)** model and **a T4 GPU**, although there might be some slight variation depending on the state of Colab.

In [9]:
# Running ASR.
speech_recognition(model_name=model_name, input_audio=input_audio_array, output_subtitle_path=output_subtitle_path, decode_options=decode_options, cache_dir=cache_dir)

=============== Loading Whisper-medium ===============


100%|██████████████████████████████████████| 1.42G/1.42G [00:12<00:00, 122MiB/s]


Begin to utilize Whisper-medium to transcribe the audio.


100%|██████████| 104500/104500 [02:39<00:00, 655.08frames/s]


The process of speech recognition costs 191.77863073349 seconds.


370it [00:00, 161185.34it/s]


=============== Saving the subtitle to ./output-信號與人生.srt ===============


You can check the result of automatic speech recognition.

In [10]:
''' Open the SRT file and read its content.
The format of SRT is:

[Index]
[Begin time] (hour:minute:second) --> [End time] (hour:minute:second)
[Transcription]

'''

with open(output_subtitle_path, 'r', encoding='utf-8') as file:
    content = file.read()

print(content)

1
00:00:00,000 --> 00:00:04,000
每次說這個學問是做出來的

2
00:00:06,000 --> 00:00:08,000
什麼意思?

3
00:00:08,000 --> 00:00:12,000
要做才會獲得學問

4
00:00:13,000 --> 00:00:16,000
你如果每天光是坐在那裡聽

5
00:00:17,000 --> 00:00:20,000
學問很可能是左耳進右耳出的

6
00:00:21,000 --> 00:00:23,000
你光是坐在那兒讀

7
00:00:23,000 --> 00:00:26,000
學問可能從眼睛進入腦海之後就忘掉了

8
00:00:26,000 --> 00:00:29,000
如何能夠學問在腦海裡面

9
00:00:31,000 --> 00:00:33,000
真的變成你自己學問

10
00:00:33,000 --> 00:00:35,000
就是要做

11
00:00:36,000 --> 00:00:39,000
可能有很多同學有這個經驗

12
00:00:39,000 --> 00:00:41,000
你如果去修某一門課

13
00:00:41,000 --> 00:00:44,000
或者做某一個實驗

14
00:00:44,000 --> 00:00:47,000
在期末就是要教一個final project

15
00:00:48,000 --> 00:00:50,000
那個final project就是要你把

16
00:00:51,000 --> 00:00:53,000
學到的很多東西

17
00:00:53,000 --> 00:00:56,000
最後整合在你的final project裡面

18
00:00:56,000 --> 00:00:58,000
最後做出來的時候

19
00:00:58,000 --> 00:01:00,000
就是把它們都整合了

20
00:01:00,000 --> 00:01:02,000
當你學期結束

21
00:01:02,000 --> 00:01:04,000
真的把final project做完的時候

22
00:01:04,000 --> 00:01:05,00

# Part3 - Preprocess the results of automatic speech recognition

In [11]:
def extract_and_save_text(srt_filename, output_filename):

    '''
    (1) Objective:
        - This function extracts the text from an SRT file and saves it to a new text file.
        - It also converts the Simplified Chinese to Traditional Chinese.

    (2) Arguments:

        - srt_filename: The path to the SRT file.

        - output_filename: The name of the output text file.

    (3) Example:
        - If your SRT file is named 'subtitle.srt' and you want to save the extracted text to a file named 'output.txt', you can use the function like this:
            extract_and_save_text('subtitle.srt', 'output.txt')

    '''

    # Open the SRT file and read its content.
    with open(srt_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # Use regular expression to remove the timecode.
    pure_text = re.sub(r'\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n', '', content)

    # Remove the empty lines.
    pure_text = re.sub(r'\n\n+', '\n', pure_text)

    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    pure_text_conversion = cc.convert(pure_text)

    # Write the extracted text to a new file.
    with open(output_filename, 'w', encoding='utf-8') as output_file:
        output_file.write(pure_text_conversion)

    print(f'Extracted text has been saved to {output_filename}.\n\n')

    return pure_text_conversion

In [12]:
def chunk_text(text, max_length):
    """
    (1) Objective:
        - This function is used to split a long string into smaller strings of a specified length.

    (2) Arguments:
        - text: str, the long string to be split.
        - max_length: int, the maximum length of each smaller string.

    (3) Returns:
        - split_text: list, a list of smaller strings.

    (3) Example:
        - If you want to split a string named "long_string" into smaller strings of length 100, you can use the function like this:
            chunk_text(long_string, 100)

    """

    return textwrap.wrap(text, max_length)

In [13]:
''' In this block, you can modify your desired parameters and the path of input file. '''

# # The length of the text chunks.
chunk_length = 512

In [14]:
# Extracts the text from an SRT file and saves it to a new text file
pure_text = extract_and_save_text(srt_filename=output_subtitle_path, output_filename=output_raw_text_path)

# Split a long document into smaller chunks of a specified length
chunks = chunk_text(text=pure_text, max_length=512)

# You can see the number of words and contents in each paragraph.
print("Review the results of splitting the long text into several short texts.\n")
for index, chunk in enumerate(chunks):
    if index == 0:
        print(f"\n========== The {index + 1}-st segment of the split ({len(chunk)} words) ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    elif index == 1:
        print(f"\n========== The {index + 1}-nd segment of the split ({len(chunk)} words) ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    elif index == 2:
        print(f"\n========== The {index + 1}-rd segment of the split ({len(chunk)} words) ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    else:
        print(f"\n========== The {index + 1}-th segment of the split ({len(chunk)} words) ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")

Extracted text has been saved to ./output-信號與人生.txt.


Review the results of splitting the long text into several short texts.


========== The 1-st segment of the split (505 words) ==========


每次說這個學問是做出來的 什麼意思? 要做才會獲得學問 你如果每天光是坐在那裡聽 學問很可能是左耳進右耳出的 你光是坐在那兒讀

學問可能從眼睛進入腦海之後就忘掉了 如何能夠學問在腦海裡面 真的變成你自己學問 就是要做 可能有很多同學有這個經驗 你如果去修某一門課 或者做某一個實驗

在期末就是要教一個final project 那個final project就是要你把 學到的很多東西 最後整合在你的final project裡面

最後做出來的時候 就是把它們都整合了 當你學期結束 真的把final project做完的時候 你會忽然發現 我真的學到很多東西 那就是做出來的學問

也許可以舉另外一個例子 就是你如果學了某一些很複雜的演算法 或者什麼 好像覺得那些不見得在你的腦海裡 可是後來老師出了個習題 那個習題教你寫一個很大的程式

要把所有東西都包進去 當你把這個程式寫完的時候你會發現 你忽然把演算法裡所有東西都弄通了 那就是學問是做出來的 所以我們永遠要記得 盡量多動手多做

在動手跟做的過程之中 學問纔可以變成是自己的 同樣的情形就是說 很多時候這樣動手或者做的表現或者成績 沒有一個成績單上的數字


========== The 2-nd segment of the split (506 words) ==========


使得很多人覺得那不重要 很多人甚至覺得這門課要做final project 我就不修了太累了 或者說那門課需要怎麼樣怎麼樣太累 我就不要做了

而不知道其實那個纔是讓你做的機會 然後可以學到最多 也就是說雖然很可能那麼辛苦的做很多事 沒有讓你獲得什麼具體成績 對你的overfitting可能沒有幫助

可是對你的全面學習是很有幫助 是該學的 那不要漏掉這些事 那這是我所說的 那這個課業內可以做的這些事 那剛才我們講到思考的時候 我覺得我漏掉一點


# Part4 - Summarization


## **You only need to choose one of the following parts.**

## **If you want to use ChatGPT, begin with this part.**
##### (1) You can refer to https://shorturl.at/X0NDY (Page 44) for obtaining ChatGPT API key.
##### (2) You can refer to https://platform.openai.com/docs/models/overview for more details about models you can use.

In [17]:
def summarization(client, summarization_prompt, model_name="gpt-3.5-turbo", temperature=0.0, top_p=1.0, max_tokens=512):
    """
    (1) Objective:
        - Use the OpenAI Chat API to summarize a given text.

    (2) Arguments:
        - client: OpenAI Chat API client.
        - summarization_prompt: The summarization prompt including the text which need to be summarized.
        - model_name: The model name, default is "gpt-3.5-turbo". You can refer to "https://platform.openai.com/docs/models/overview" for more details.
        - temperature: Controls randomness in the response. Lower values make responses more deterministic, default is 0.0.
        - top_p: Controls diversity via nucleus sampling. Higher values lead to more diverse responses, default is 1.0.
        - max_tokens: The maximum number of tokens to generate in the completion, default is 512.

    (3) Return:
        - The summarized text.

    (4) Example:
        - If the text is "ABC" and the summarization prompt is "DEF", system_prompt is "GHI", model_name is "gpt-3.5-turbo",
          temperature is 0.0, top_p is 1.0, and max_tokens is 512, then you can call the function like this:

              summarization(client=client, text="ABC", summarization_prompt="DEF", system_prompt="GHI", model_name="gpt-3.5-turbo", temperature=0.0, top_p=1.0, max_tokens=512)

    """

    # The user prompt is a concatenation of the summarization_prompt and text.
    user_prompt = summarization_prompt

    while True:

        try:
            # Use the OpenAI Chat API to summarize the text.
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": user_prompt,
                    }
                ],
                    model=model_name,
                    temperature=temperature,
                    top_p=top_p,
                    max_tokens=max_tokens
            )

            break

        except:
            # If the API call fails, wait for 1 second and try again.
            print("The API call fails, wait for 1 second and try again.")
            time.sleep(1)

    return chat_completion.choices[0].message.content

In [47]:
# @title Parameter Setting of ChatGPT { run: "auto" }
''' ===== In this block, you can modify your desired parameters and set your OpenAI API key ===== '''

# Your OpenAI API key.
# @markdown **openai_api_key**: Your OpenAI API key.
openai_api_key = "YOUR_GHATGPT_API_KEY" # @param {type:"string"}

# The model name, default is "gpt-3.5-turbo". You can refer to "https://platform.openai.com/docs/models/overview" for more details.
# @markdown **model_name**: The model name, default is "gpt-3.5-turbo". You can refer to "https://platform.openai.com/docs/models/overview" for more details.
model_name = "gpt-3.5-turbo" # @param {type: "string"}

# Controls randomness in the response. Lower values make responses more deterministic.
# @markdown **temperature**: Controls randomness in the response. Lower values make responses more deterministic.
temperature = 0 # @param {type:"slider", min:0, max:1, step:0.1}

# Controls diversity via nucleus sampling. Higher values lead to more diverse responses.
# @markdown **top_p**: Controls diversity via nucleus sampling. Higher values lead to more diverse responses.
top_p = 0 # @param {type:"slider", min:0, max:1, step:0.1}

In [23]:
# Construct openai client.
client = OpenAI(api_key=openai_api_key)

The code block below takes about **30** seconds to run when using the **gpt-3.5-turbo** model, but the actual time may vary depending on the condition of Colab and the status of the OpenAI API.

### We offer the following two methods for summarization.
Reference: https://reurl.cc/VzagLA

#### **If you want to use the method of Multi-Stage Summarization, begin with this part.**


In [24]:
# @title Prompt Setting of ChatGPT Multi-Stage Summarization: Paragraph { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# The maximum number of tokens to generate in the completion.
# @markdown **max_tokens**: The maximum number of tokens to generate in the completion.
max_tokens = 350 # @param {type:"integer"}

# @markdown #### Changing **summarization_prompt_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.
summarization_prompt_template = "用 300 個字內寫出這段文字的摘要，其中包括要點和所有重要細節：<text>" # @param {type:"string"}

##### Step1: Split the long text into multiple smaller pieces and obtain summaries for each smaller text piece separately

The code block below takes about **80** seconds to run when using the (1) **gpt-3.5-turbo** model, (2) length of chunks is 512 and (3) maximum number of tokens is 250, but the actual time may vary depending on the condition of Colab and the status of the OpenAI API.

In [25]:
paragraph_summarizations = []

# First, we summarize each section that has been split up separately.
for index, chunk in enumerate(chunks):

    # Record the start time.
    start = time.time()

    # Construct summarization prompt.
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

    # We summarize each section that has been split up separately.
    response = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

    # Calculate the execution time and round it to 2 decimal places.
    cost_time = round(time.time() - start, 2)

    # Print the summary and its length.
    print(f"----------------------------Summary of Segment {index + 1}----------------------------\n")
    for text in textwrap.wrap(response, 80):
        print(f"{text}\n")
    print(f"Length of summary for segment {index + 1}: {len(response)}")
    print(f"Time taken to generate summary for segment {index + 1}: {cost_time} sec.\n")

    # Record the result.
    paragraph_summarizations.append(response)

----------------------------Summary of Segment 1----------------------------

學問是需要透過實際做來獲得的，單純聽或讀可能無法真正吸收知識。舉例來說，修課最後的final project能幫助整合所學，讓知識真正被掌握。另外，寫程式也能幫

助理解複雜演算法，透過實際操作才能真正理解。因此，我們應該記得多動手多做，只有在實際操作中，學問才能成為自己的。成績單上的數字並不能完全反映學問的深度。

Length of summary for segment 1: 156
Time taken to generate summary for segment 1: 3.45 sec.

----------------------------Summary of Segment 2----------------------------

許多人認為某些課程不重要，甚至選擇不修讀因為覺得太辛苦或無法獲得具體成績。然而，這些課程可能提供最多學習機會，對全面學習有幫助。重要的是不要錯過這些機會，培養自

己的思考能力和習慣。透過閱讀數學式子和思考課程內容，可以深入理解並提升思考能力。這是最重要的學習方法，需要花費大量時間和精力。

Length of summary for segment 2: 143
Time taken to generate summary for segment 2: 3.4 sec.

----------------------------Summary of Segment 3----------------------------

學習不僅僅是在課業內，還包括課業外的各種活動。學習是一種增長、進步和獲得快樂的過程。例如，打球可以提升健康、手腦協調、團隊精神和人際互動能力，是一個很好的學習機

會。爬山和旅行也是很好的學習機會，可以增長見識、擴展視野，並帶來快樂。無論是什麼活動，只要讓你感到增長、進步和快樂，都值得花時間和精力去學習。

Length of summary for segment 3: 151
Time taken to generate summary for segment 3: 4.05 sec.

-----------------------

In [26]:
# First, we collect all the summarizations obtained before and print them.

collected_summarization = ""
for index, paragraph_summarization in enumerate(paragraph_summarizations):
    collected_summarization += f"Summary of segment {index + 1}: {paragraph_summarization}\n"

print(collected_summarization)

Summary of segment 1: 學問是需要透過實際做來獲得的，單純聽或讀可能無法真正吸收知識。舉例來說，修課最後的final project能幫助整合所學，讓知識真正被掌握。另外，寫程式也能幫助理解複雜演算法，透過實際操作才能真正理解。因此，我們應該記得多動手多做，只有在實際操作中，學問才能成為自己的。成績單上的數字並不能完全反映學問的深度。
Summary of segment 2: 許多人認為某些課程不重要，甚至選擇不修讀因為覺得太辛苦或無法獲得具體成績。然而，這些課程可能提供最多學習機會，對全面學習有幫助。重要的是不要錯過這些機會，培養自己的思考能力和習慣。透過閱讀數學式子和思考課程內容，可以深入理解並提升思考能力。這是最重要的學習方法，需要花費大量時間和精力。
Summary of segment 3: 學習不僅僅是在課業內，還包括課業外的各種活動。學習是一種增長、進步和獲得快樂的過程。例如，打球可以提升健康、手腦協調、團隊精神和人際互動能力，是一個很好的學習機會。爬山和旅行也是很好的學習機會，可以增長見識、擴展視野，並帶來快樂。無論是什麼活動，只要讓你感到增長、進步和快樂，都值得花時間和精力去學習。
Summary of segment 4: 談戀愛和交朋友都是學習人際互動和溝通的好機會，即使沒有緣分也可以透過交朋友來學習。在大學裡，有很多機會參加各種活動，例如戲劇社或舞蹈社，這些活動可以幫助我們成長和進步。即使不是表演者，參與幕後工作也能獲得寶貴的經驗。因此，大家應該積極參與各種活動，透過互動和合作來提升自己。
Summary of segment 5: 這段文字談論到在學習過程中，除了專業知識外，參與各種活動和團隊合作也是非常重要的。這些活動可以幫助人們成長和進步，並提供寶貴的學習機會。即使這些活動沒有考試或成績，但它們對個人發展和職業目標都有著重要的影響。在電機工程領域，成功很少是一個人能夠完成的，需要與他人合作。因此，學習如何進入團隊、成為領導者以及推動想法的能力都是非常重要的。這些被稱為軟實力，是除了專業技能外的另一種重要能力。
Summary of segment 6: 成功與否不僅取決於硬實力，更重要的是軟實力，包括人際交往能力、溝通能力、領導能力等。這些soft skills對於成功至關重要，並且可以透過課外活動和學習機會來培養

##### Step2: After obtaining summaries for each smaller text piece separately, process these summaries to generate the final summary.

In [36]:
# @title Prompt Setting of ChatGPT Multi-Stage Summarization: Total { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# We set the maximum number of tokens to ensure that the final summary does not exceed 550 tokens.
# @markdown **max_tokens**: We set the maximum number of tokens to ensure that the final summary does not exceed 550 tokens.
max_tokens = 550 # @param {type:"integer"}

# @markdown ### Changing **summarization_prompt_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.
summarization_prompt_template = "使用繁體中文在 500 字以內寫出以下文字的簡潔摘要：<text>" # @param {type:"string"}

The code block below takes about **10** seconds to run when using the (1) **gpt-3.5-turbo** model and (2) maximum number of tokens is 500, but the actual time may vary depending on the condition of Colab and the status of the OpenAI API.

In [37]:
# Finally, we compile a final summary from the summaries of each section.

# Record the start time.
start = time.time()

# Run final summarization.
summarization_prompt = summarization_prompt_template.replace("<text>", collected_summarization)
final_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

# Calculate the execution time and round it to 2 decimal places.
cost_time = round(time.time() - start, 2)

# Print the summary and its length.
print(f"----------------------------Final Summary----------------------------\n")
for text in textwrap.wrap(final_summarization, 80):
        print(f"{text}")
print(f"\nLength of final summary: {len(final_summarization)}")
print(f"Time taken to generate the final summary: {cost_time} sec.")

----------------------------Final Summary----------------------------

總結第一部分：學問需要透過實際做來獲得，單純聽或讀可能無法真正吸收知識。修課最後的final
project和寫程式能幫助整合所學，讓知識真正被掌握。成績單上的數字並不能完全反映學問的深度。  總結第二部分：某些課程可能提供最多學習機會，對全面學習有幫助
。重要的是不要錯過這些機會，培養自己的思考能力和習慣。透過閱讀數學式子和思考課程內容，可以深入理解並提升思考能力。
總結第三部分：學習不僅僅是在課業內，還包括課業外的各種活動。打球、爬山和旅行等活動都是很好的學習機會，可以增長見識、擴展視野，並帶來快樂。
總結第四部分：談戀愛和交朋友都是學習人際互動和溝通的好機會，參與各種活動可以幫助我們成長和進步。大家應該積極參與各種活動，透過互動和合作來提升自己。
總結第五部分：參與各種活動和團隊合作是非常重要的，即使沒有考試或成績，這些活動對個人發展和職業目標都有著重要的影響。學

Length of final summary: 392
Time taken to generate the final summary: 6.94 sec.


In [38]:
''' In this block, you can modify your desired output path of final summary. '''

output_path = f"./final-summary-{suffix}-chatgpt-multi-stage.txt"

# If you need to convert Simplified Chinese to Traditional Chinese, please set this option to True; otherwise, set it to False.
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    final_summarization = cc.convert(final_summarization)

# Output your final summary
with open(output_path, "w") as fp:
    fp.write(final_summarization)

# Show the result.
print(f"Final summary has been saved to {output_path}")
print(f"\n===== Below is the final summary ({len(final_summarization)} words) =====\n")
for text in textwrap.wrap(final_summarization, 64):
    print(text)

Final summary has been saved to ./final-summary-信號與人生-chatgpt-multi-stage.txt

===== Below is the final summary (392 words) =====

總結第一部分：學問需要透過實際做來獲得，單純聽或讀可能無法真正吸收知識。修課最後的final
project和寫程式能幫助整合所學，讓知識真正被掌握。成績單上的數字並不能完全反映學問的深度。  總結第二部分：某些課程可能提
供最多學習機會，對全面學習有幫助。重要的是不要錯過這些機會，培養自己的思考能力和習慣。透過閱讀數學式子和思考課程內容，可以深入理
解並提升思考能力。  總結第三部分：學習不僅僅是在課業內，還包括課業外的各種活動。打球、爬山和旅行等活動都是很好的學習機會，可以
增長見識、擴展視野，並帶來快樂。  總結第四部分：談戀愛和交朋友都是學習人際互動和溝通的好機會，參與各種活動可以幫助我們成長和進
步。大家應該積極參與各種活動，透過互動和合作來提升自己。
總結第五部分：參與各種活動和團隊合作是非常重要的，即使沒有考試或成績，這些活動對個人發展和職業目標都有著重要的影響。學


#### **If you want to use the method of Refinement, begin with this part.**



In [39]:
# @title Prompt Setting of ChatGPT Refinement { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# We set the maximum number of tokens.
# @markdown **max_tokens**: We set the maximum number of tokens.
max_tokens = 550 # @param {type:"integer"}

# @markdown ### Changing **summarization_prompt_template** and **summarization_prompt_refine_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.

# Initial prompt.
# @markdown **summarization_prompt_template**: Initial prompt.
summarization_prompt_template = "請在 300 字以內，提供以下文字的簡潔摘要:<text>" # @param {type:"string"}

# Refinement prompt.
# @markdown **summarization_prompt_refinement_template**: Refinement prompt.
summarization_prompt_refinement_template = "請在 500 字以內，結合原先的摘要和新的內容，提供簡潔的摘要:<text>" # @param {type:"string"}

The code block below takes about **200** seconds to run when using the (1) **gpt-3.5-turbo** model and (2) maximum number of tokens is 500, but the actual time may vary depending on the condition of Colab and the status of the OpenAI API.

Pipeline of the method of Refinement.

Step1: It starts by running a prompt on a small portion of the data, generating initial output.

Step2: For each following document, the previous output is fed in along with the new document.

Step3: The LLM is instructed to refine the output based on the new document's information.

Step4: This process continues iteratively until all documents have been processed.

In [40]:
paragraph_summarizations = []

# First, we summarize each section that has been split up separately.
for index, chunk in enumerate(chunks):

    if index == 0:
        # Record the start time.
        start = time.time()

        # Construct summarization prompt.
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

        # Step1: It starts by running a prompt on a small portion of the data, generating initial output.
        first_paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # Record the result.
        paragraph_summarizations.append(first_paragraph_summarization)

        # Calculate the execution time and round it to 2 decimal places.
        cost_time = round(time.time() - start, 2)

        # Print the summary and its length.
        print(f"----------------------------Summary of Segment {index + 1}----------------------------\n")
        for text in textwrap.wrap(response, 80):
            print(f"{text}\n")
        print(f"Length of summary for segment {index + 1}: {len(response)}")
        print(f"Time taken to generate summary for segment {index + 1}: {cost_time} sec.\n")


    else:
        # Record the start time.
        start = time.time()

        # Step2: For each following document, the previous output is fed in along with the new document.
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的內容: {chunk}"""

        # Construct refinement prompt for summarization.
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_text)

        # Step3: The LLM is instructed to refine the output based on the new document's information.
        paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # Record the result.
        paragraph_summarizations.append(paragraph_summarization)

        # Calculate the execution time and round it to 2 decimal places.
        cost_time = round(time.time() - start, 2)

        # print results.
        print(f"----------------------------Summary of the First {index + 1} Segments----------------------------\n")
        for text in textwrap.wrap(paragraph_summarization, 80):
            print(f"{text}\n")
        print(f"Length of summary for the first {index + 1} segments: {len(paragraph_summarization)}")
        print(f"Time taken to generate summary for the first {index + 1} segments: {cost_time} sec.\n")

    # Step4: This process continues iteratively until all documents have been processed.

----------------------------Summary of Segment 1----------------------------

我希望在未來的5年、10年、15年甚至更長的時間裡，將我的努力集中在實現一些非常重要的事情上。這些事情是我非常想要做的，也是我認為具有意義的。這就是我的長程目標

。對於那些能夠想清楚自己的長程目標的人來說，他們更容易朝著目標前進。我認為這四件事情是真正影響我未來的關鍵。

Length of summary for segment 1: 134
Time taken to generate summary for segment 1: 2.87 sec.

----------------------------Summary of the First 2 Segments----------------------------

學問是做出來的，透過實際動手做final project或寫程式等方式，將所學知識應用整合，才能真正理解和掌握。儘管有些人可能覺得這些做法太辛苦，並且可能不會立

即反映在成績上，但這些實作方式卻是讓你學到最多、全面學習的機會。在學習的過程中，不要漏掉這些重要的事情，培養自己的思考能力和習慣。透過思考數學式子和閱讀的同時，

深入理解知識背後的意義，這才是最重要的。因此，記得多動手多做，讓學問變成自己的。成績不一定代表真正的學習成果，重要的是你如何運用所學來提升自己的能力和思考水平。

Length of summary for the first 2 segments: 240
Time taken to generate summary for the first 2 segments: 5.44 sec.

----------------------------Summary of the First 3 Segments----------------------------

學問是透過實際動手做final project或寫程式等方式，將所學知識應用整合，才能真正理解和掌握。儘管有些人可能覺得這些做法太辛苦，並且可能不會立即反映在成

績上，但這些實作方式卻是讓你學到最多、全面學習的機會。在學習的過程中，不要漏掉這些重要的事情，培養自己的思考能力和習慣。透過思考數學式子

In [41]:
''' In this block, you can modify your desired output path of final summary. '''

output_path = f"./final-summary-{suffix}-chatgpt-refinement.txt"

# If you need to convert Simplified Chinese to Traditional Chinese, please set this option to True; otherwise, set it to False.
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    paragraph_summarizations[-1] = cc.convert(paragraph_summarizations[-1])

# Output your final summary
with open(output_path, "w") as fp:
    fp.write(paragraph_summarizations[-1])

# Show the result.
print(f"Final summary has been saved to {output_path}")
print(f"\n===== Below is the final summary ({len(paragraph_summarizations[-1])} words) =====\n")
for text in textwrap.wrap(paragraph_summarizations[-1], 80):
    print(text)

Final summary has been saved to ./final-summary-信號與人生-chatgpt-refinement.txt

===== Below is the final summary (397 words) =====

在電機工程領域的職業發展中，35歲到55歲被視為黃金時代，這段時間是最好的發展時期。不同人的發展軌跡可能有所不同，但實力、努力、大智和自我技能是影響發展的主要因
素。成功的電機工程師通常擁有豐富的軟實力，這是他們成功的關鍵之一。在學習的過程中，參與各種課外活動和經歷是非常重要的，這些活動能讓人成長和進步，同時提升人際互動
和溝通能力。除了專業知識外，培養軟實力和全面發展自己的能力對於電機工程師的職業發展至關重要。實力、努力、大智和自我技能是影響電機工程師職業發展的主要因素，而全面
的學習和參與各種課外活動能夠提升個人的軟實力，進而影響整體發展。建立長程目標也是重要的，每個人都可以思考自己想要達成的目標，並在適當的時機開始努力實現。這些因素
共同影響著電機工程師在職業生涯中的成功與成長。  此外，設定長程目標對於個人的職業發展也是至關重要的。無論是花費5年、10年、15年甚至更長的時間，將努力


## **If you want to use Gemini, begin with this part.**
##### (1) You can refer to https://shorturl.at/X0NDY (Page 35) for obtaining Gemini API key.
##### (2) You can refer to https://ai.google.dev/models/gemini for more details about which models you can use.

In [ ]:
def summarization(summarization_prompt, model_name="gemini-pro", temperature=0.0, top_p=1.0, max_tokens=512):
    """
    (1) Objective:
        - Use the OpenAI Chat API to summarize a given text.

    (2) Arguments:
        - summarization_prompt: The summarization prompt.
        - model_name: The model name, default is "gemini-pro". You can refer to "https://ai.google.dev/models/gemini" for more details.
        - temperature: Controls randomness in the response. Lower values make responses more deterministic, default is 0.0.
        - top_p: Controls diversity via nucleus sampling. Higher values lead to more diverse responses, default is 1.0.
        - max_tokens: The maximum number of tokens to generate in the completion, default is 512.

    (3) Return:
        - The summarized text.

    (4) Example:
        - If the text is "ABC" and the summarization prompt is "DEF", model_name is "gemini-pro",
          temperature is 0.0, top_p is 1.0, and max_tokens is 512, then you can call the function like this:

              summarization(text="ABC", summarization_prompt="DEF", model_name="gemini-pro", temperature=0.0, top_p=1.0, max_tokens=512)

    """

    # The user prompt is a concatenation of the summarization_prompt and text.
    user_prompt = summarization_prompt

    # Load the generative model.
    model = genai.GenerativeModel(model_name)

    # Set the generation configuration.
    generation_config = genai.GenerationConfig(temperature=temperature, top_p=top_p, max_output_tokens=max_tokens)

    while True:

        try:
            # Use the OpenAI Chat API to summarize the text.
            response = model.generate_content(contents=user_prompt, generation_config=generation_config)

            break

        except:
            # If the API call fails, wait for 1 second and try again.
            print("The API call fails, wait for 1 second and try again.")
            time.sleep(1)

    return response.text

In [ ]:
# @title Parameter Setting of Gemini { run: "auto" }
''' In this block, you can modify your desired parameters and set your api key. '''

# Your google api key.
# @markdown **google_api_key**: Your google api key.
google_api_key = "YOUR_GOOGLE_API_KEY" # @param {type:"string"}

# The model name. You can refer to "https://ai.google.dev/models/gemini" for more details.
# @markdown **model_name**: The model name. You can refer to "https://ai.google.dev/models/gemini" for more details.
model_name = "gemini-pro" # @param {type:"string"}

# Controls randomness in the response. Lower values make responses more deterministic
# @markdown **temperature**: Controls randomness in the response. Lower values make responses more deterministic.
temperature = 0.0 # @param {type:"slider", min:0, max:1, step:0.1}

# Controls diversity via nucleus sampling. Higher values lead to more diverse responses
# @markdown **top_p**: Controls diversity via nucleus sampling. Higher values lead to more diverse responses.
top_p = 1.0 # @param {type:"slider", min:0, max:1, step:0.1}

In [ ]:
# Set Google API key.
genai.configure(api_key=google_api_key)

### We offer the following two methods for summarization.
Reference: https://reurl.cc/VzagLA

#### **If you want to use the method of Multi-Stage Summarization, begin with this part.**


In [ ]:
# @title Prompt Setting of Gemini Multi-Stage Summarization: Paragraph { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# The maximum number of tokens to generate in the completion.
# @markdown **max_tokens**: The maximum number of tokens to generate in the completion.
max_tokens = 350 # @param {type:"integer"}

# @markdown #### Changing **summarization_prompt_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.
summarization_prompt_template = "用 300 個字內寫出這段文字的摘要，其中包括要點和所有重要細節：<text>" # @param {type:"string"}

##### Step1: Split the long text into multiple smaller pieces and obtain summaries for each smaller text piece separately


The code block below takes about **40** seconds to run when using the (1) **gemini-pro** model, (2) length of chunks is 512 and (3) maximum number of tokens is 350, but the actual time may vary depending on the condition of Colab and the status of the Google API.

In [ ]:
paragraph_summarizations = []

# First, we summarize each section that has been split up separately.
for index, chunk in enumerate(chunks):

    # Record the start time.
    start = time.time()

    # Construct summarization prompt.
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

    # We summarize each section that has been split up separately.
    response = summarization(summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

    # Calculate the execution time and round it to 2 decimal places.
    cost_time = round(time.time() - start, 2)

    # Print the summary and its length.
    print(f"----------------------------Summary of Segment {index + 1}----------------------------\n")
    for text in textwrap.wrap(response, 80):
        print(f"{text}\n")
    print(f"Length of summary for segment {index + 1}: {len(response)}")
    print(f"Time taken to generate summary for segment {index + 1}: {cost_time} sec.\n")

    # Record the result.
    paragraph_summarizations.append(response)

In [ ]:
# First, we collect all the summarizations obtained before and print them.

collected_summarization = ""
for index, paragraph_summarization in enumerate(paragraph_summarizations):
    collected_summarization += f"Summary of segment {index + 1}: {paragraph_summarization}\n"

print(collected_summarization)

#####Step2: After obtaining summaries for each smaller text piece separately, process these summaries to generate the final summary.

In [ ]:
# @title Prompt Setting of Gemini Multi-Stage Summarization: Total { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# We set the maximum number of tokens to ensure that the final summary does not exceed 550 tokens.
# @markdown **max_tokens**: We set the maximum number of tokens to ensure that the final summary does not exceed 550 tokens.
max_tokens = 550 # @param {type:"integer"}

# @markdown ### Changing **summarization_prompt_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.
summarization_prompt_template = "在 500 字以內寫出以下文字的簡潔摘要：<text>" # @param {type:"string"}

The code block below takes about **20** seconds to run when using the (1) **gemini-pro** model, (2) length of chunks is 512 and (3) maximum number of tokens is 550, but the actual time may vary depending on the condition of Colab and the status of the Google API.

In [ ]:
# Finally, we compile a final summary from the summaries of each section.

# Record the start time.
start = time.time()

# Run final summarization.
summarization_prompt = summarization_prompt_template.replace("<text>", collected_summarization)
final_summarization = summarization(summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

# Calculate the execution time and round it to 2 decimal places.
cost_time = round(time.time() - start, 2)

# Print the summary and its length.
print(f"----------------------------Final Summary----------------------------\n")
for text in textwrap.wrap(final_summarization, 80):
        print(f"{text}")
print(f"\nLength of final summary: {len(final_summarization)}")
print(f"Time taken to generate the final summary: {cost_time} sec.")

In [ ]:
''' In this block, you can modify your desired output path of final summary. '''

output_path = f"./final-summary-{suffix}-gemini-multi-stage.txt"

# If you need to convert Simplified Chinese to Traditional Chinese, please set this option to True; otherwise, set it to False.
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    final_summarization = cc.convert(final_summarization)

# Output your final summary
with open(output_path, "w") as fp:
    fp.write(final_summarization)

print(f"Final summary has been saved to {output_path}")

#### **If you want to use the method of Refinement, begin with this part.**



In [ ]:
# @title Prompt Setting of Gemini Refinement { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# We set the maximum number of tokens.
# @markdown **max_tokens**: We set the maximum number of tokens.
max_tokens = 550 # @param {type:"integer"}

# @markdown ### Changing **summarization_prompt_template** and **summarization_prompt_refine_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.

# Initial prompt.
# @markdown **summarization_prompt_template**: Initial prompt.
summarization_prompt_template = "請在 300 字以內，提供以下文字的簡潔摘要:<text>" # @param {type:"string"}

# Refinement prompt.
# @markdown **summarization_prompt_refinement_template**: Refinement prompt.
summarization_prompt_refinement_template = "請在 500 字以內，結合原先的摘要和新的內容，提供簡潔的摘要:<text>" # @param {type:"string"}

Pipeline of the method of Refinement.

Step1: It starts by running a prompt on a small portion of the data, generating initial output.

Step2: For each following document, the previous output is fed in along with the new document.

Step3: The LLM is instructed to refine the output based on the new document's information.

Step4: This process continues iteratively until all documents have been processed.

The code block below takes about **45** seconds to run when using the (1) **gemini-pro** model and (2) maximum number of tokens is 500, but the actual time may vary depending on the condition of Colab and the status of the Google API.

In [ ]:
paragraph_summarizations = []

# First, we summarize each section that has been split up separately.
for index, chunk in enumerate(chunks):

    if index == 0:
        # Record the start time.
        start = time.time()

        # Construct summarization prompt.
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

        # Step1: It starts by running a prompt on a small portion of the data, generating initial output.
        first_paragraph_summarization = summarization(summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # Record the result.
        paragraph_summarizations.append(first_paragraph_summarization)

        # Calculate the execution time and round it to 2 decimal places.
        cost_time = round(time.time() - start, 2)

        # Print the summary and its length.
        print(f"----------------------------Summary of Segment {index + 1}----------------------------\n")
        for text in textwrap.wrap(paragraph_summarizations, 80):
            print(f"{text}\n")
        print(f"Length of summary for segment {index + 1}: {len(paragraph_summarizations)}")
        print(f"Time taken to generate summary for segment {index + 1}: {cost_time} sec.\n")

    else:
        # Record the start time.
        start = time.time()

        # Step2: For each following document, the previous output is fed in along with the new document.
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的內容: {chunk}"""

        # Construct refinement prompt for summarization.
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_text)

        # Step3: The LLM is instructed to refine the output based on the new document's information.
        paragraph_summarization = summarization(summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # Record the result.
        paragraph_summarizations.append(paragraph_summarization)

        # Calculate the execution time and round it to 2 decimal places.
        cost_time = round(time.time() - start, 2)

        # print results.
        print(f"----------------------------Summary of the First {index + 1} Segments----------------------------\n")
        for text in textwrap.wrap(paragraph_summarization, 80):
            print(f"{text}\n")
        print(f"Length of summary for the first {index + 1} segments: {len(paragraph_summarization)}")
        print(f"Time taken to generate summary for the first {index + 1} segments: {cost_time} sec.\n")

    # Step4: This process continues iteratively until all documents have been processed.

In [ ]:
''' In this block, you can modify your desired output path of final summary. '''

output_path = f"./final-summary-{suffix}-gemini-refinement.txt"

# If you need to convert Simplified Chinese to Traditional Chinese, please set this option to True; otherwise, set it to False.
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    paragraph_summarizations[-1] = cc.convert(paragraph_summarizations[-1])

# Output your final summary
with open(output_path, "w") as fp:
    fp.write(paragraph_summarizations[-1])

# Show the result.
print(f"Final summary has been saved to {output_path}")
print(f"\n===== Below is the final summary ({len(paragraph_summarizations[-1])} words) =====\n")
for text in textwrap.wrap(paragraph_summarizations[-1], 64):
    print(text)

## **If you want to use Claude, begin with this part.**
##### (1) You can refer to https://reurl.cc/yLy06D for obtaining Claude API key.
##### (2) You can refer to https://docs.anthropic.com/claude/docs/models-overview for more details about which models you can use.

In [ ]:
def summarization(client, summarization_prompt, model_name="claude-3-sonnet-20240229", temperature=0.0, top_p=1.0, max_tokens=512):
    """
    (1) Objective:
        - Use the Claude API to summarize a given text.

    (2) Arguments:
        - client: Claude API client.
        - text: The text to be summarized.
        - summarization_prompt: The summarization prompt.
        - model_name: The model name, default is "claude-3-sonnet-20240229". You can refer to "https://docs.anthropic.com/claude/docs/models-overview#model-comparison" for more details.
        - temperature: Controls randomness in the response. Lower values make responses more deterministic, default is 0.0.
        - top_p: Controls diversity via nucleus sampling. Higher values lead to more diverse responses, default is 1.0.
        - max_tokens: The maximum number of tokens to generate in the completion, default is 512.

    (3) Return:
        - The summarized text.

    (4) Example:
        - If the text is "ABC" and the summarization prompt is "DEF", system_prompt is "GHI", model_name is "claude-3-sonnet-20240229",
          temperature is 0.0, top_p is 1.0, and max_tokens is 512, then you can call the function like this:

              summarization(client=client, text="ABC", summarization_prompt="DEF", system_prompt="GHI", model_name="claude-3-sonnet-20240229", temperature=0.0, top_p=1.0, max_tokens=512)

    """

    user_prompt = summarization_prompt

    while True:

        try:
            # Use the Claude API to summarize the text.
            message = client.messages.create(
                model=model_name,
                max_tokens=max_tokens,
                temperature=temperature,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )

            break

        except:
            # If the API call fails, wait for 1 second and try again.
            print("The API call fails, wait for 1 second and try again.")
            time.sleep(1)

    return message.content[0].text

In [ ]:
# @title Parameter Setting of Claude { run: "auto" }
''' ===== In this block, you can modify your desired parameters and set your Claude API key ===== '''

# Your Claude API key.
# @markdown **claude_api_key**: Your Claude api key.
claude_api_key = "YOUR_CLAUDE_API_KEY" # @param {type:"string"}

# The model name, default is "claude-3-opus-20240229". You can refer to "https://docs.anthropic.com/claude/docs/models-overview#model-comparison" for more details.
# @markdown **model_name**: The model name, default is "claude-3-opus-20240229". You can refer to "https://docs.anthropic.com/claude/docs/models-overview#model-comparison" for more details.
model_name = "claude-3-opus-20240229" # @param {type:"string"}

# Controls randomness in the response. Lower values make responses more deterministic.
# @markdown **temperature**: Controls randomness in the response. Lower values make responses more deterministic.
temperature = 1 # @param {type:"slider", min:0, max:1, step:0.1}

# Controls diversity via nucleus sampling. Higher values lead to more diverse responses.
# @markdown **top_p**: Controls diversity via nucleus sampling. Higher values lead to more diverse responses.
top_p = 1.0 # @param {type:"slider", min:0, max:1, step:0.1}

In [ ]:
# Construct Claude client.
client = anthropic.Anthropic(api_key=claude_api_key)

### We offer the following two methods for summarization.
Reference: https://reurl.cc/VzagLA

#### **If you want to use the method of Multi-Stage Summarization, begin with this part.**


In [ ]:
# @title Prompt Setting of Claude Multi-Stage Summarization: Paragraph { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# The maximum number of tokens to generate in the completion.
# @markdown **max_tokens**: The maximum number of tokens to generate in the completion.
max_tokens = 350 # @param {type:"integer"}

# @markdown #### Changing **summarization_prompt_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.
summarization_prompt_template = "用 300 個字內寫出這段文字的摘要，其中包括要點和所有重要細節：<text>" # @param {type:"string"}

##### Step1: Split the long text into multiple smaller pieces and obtain summaries for each smaller text piece separately

The code block below takes about **120** seconds to run when using the (1) **claude-3-opus-20240229** model, (2) length of chunks is 512 and (3) maximum number of tokens is 350, but the actual time may vary depending on the condition of Colab and the status of the Claude API.

In [ ]:
paragraph_summarizations = []

# First, we summarize each section that has been split up separately.
for index, chunk in enumerate(chunks):

    # Record the start time.
    start = time.time()

    # Construct summarization prompt.
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

    # We summarize each section that has been split up separately.
    response = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

    # Calculate the execution time and round it to 2 decimal places.
    cost_time = round(time.time() - start, 2)

    # Print the summary and its length.
    print(f"----------------------------Summary of Segment {index + 1}----------------------------\n")
    for text in textwrap.wrap(response, 80):
        print(f"{text}\n")
    print(f"Length of summary for segment {index + 1}: {len(response)}")
    print(f"Time taken to generate summary for segment {index + 1}: {cost_time} sec.\n")

    # Record the result.
    paragraph_summarizations.append(response)

In [ ]:
# First, we collect all the summarizations obtained before and print them.

collected_summarization = ""
for index, paragraph_summarization in enumerate(paragraph_summarizations):
    collected_summarization += f"Summary of segment {index + 1}: {paragraph_summarization}\n\n"

print(collected_summarization)

##### Step2: After obtaining summaries for each smaller text piece separately, process these summaries to generate the final summary.

In [ ]:
# @title Prompt Setting of Gemini Multi-Stage Summarization: Total { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# We set the maximum number of tokens to ensure that the final summary does not exceed 550 tokens.
# @markdown **max_tokens**: We set the maximum number of tokens to ensure that the final summary does not exceed 550 tokens.
max_tokens = 550 # @param {type:"integer"}

# @markdown ### Changing **summarization_prompt_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.
summarization_prompt_template = "在 500 字以內寫出以下文字的簡潔摘要：<text>" # @param {type:"string"}

The code block below takes about **25** seconds to run when using the (1) **claude-3-opus-20240229** model, (2) length of chunks is 512 and (3) maximum number of tokens is 550, but the actual time may vary depending on the condition of Colab and the status of the Claude API.

In [ ]:
# Finally, we compile a final summary from the summaries of each section.

# Record the start time.
start = time.time()

summarization_prompt = summarization_prompt_template.replace("<text>", collected_summarization)

# Run final summarization.
final_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

# Calculate the execution time and round it to 2 decimal places.
cost_time = round(time.time() - start, 2)

# Print the summary and its length.
print(f"----------------------------Final Summary----------------------------\n")
for text in textwrap.wrap(final_summarization, 80):
    print(f"{text}")
print(f"\nLength of final summary: {len(final_summarization)}")
print(f"Time taken to generate the final summary: {cost_time} sec.")

In [ ]:
''' In this block, you can modify your desired output path of final summary. '''

output_path = f"./final-summary-{suffix}-claude-multi-stage.txt"

# If you need to convert Simplified Chinese to Traditional Chinese, please set this option to True; otherwise, set it to False.
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    final_summarization = cc.convert(final_summarization)

# Output your final summary
with open(output_path, "w") as fp:
    fp.write(final_summarization)

# Show the result.
print(f"Final summary has been saved to {output_path}")
print(f"\n===== Below is the final summary ({len(final_summarization)} words) =====\n")
for text in textwrap.wrap(final_summarization, 64):
    print(text)

#### **If you want to use the method of Refinement, begin with this part.**



In [ ]:
# @title Prompt Setting of Claude Refinement { run: "auto" }
''' You can modify the summarization prompt and maximum number of tokens. '''
''' However, DO NOT modify the part of <text>.'''

# We set the maximum number of tokens.
# @markdown **max_tokens**: We set the maximum number of tokens.
max_tokens = 550 # @param {type:"integer"}

# @markdown ### Changing **summarization_prompt_template** and **summarization_prompt_refine_template**
# @markdown You can modify the summarization prompt and maximum number of tokens. However, **DO NOT** modify the part of `<text>`.

# Initial prompt.
# @markdown **summarization_prompt_template**: Initial prompt.
summarization_prompt_template = "請在 300 字以內，提供以下文字的簡潔摘要:<text>" # @param {type:"string"}

# Refinement prompt.
# @markdown **summarization_prompt_refinement_template**: Refinement prompt.
summarization_prompt_refinement_template = "請在 500 字以內，結合原先的摘要和新的內容，提供簡潔的摘要:<text>" # @param {type:"string"}

Pipeline of the method of Refinement.

Step1: It starts by running a prompt on a small portion of the data, generating initial output.

Step2: For each following document, the previous output is fed in along with the new document.

Step3: The LLM is instructed to refine the output based on the new document's information.

Step4: This process continues iteratively until all documents have been processed.

The code block below takes about **150** seconds to run when using the (1) **claude-3-opus-20240229** model and (2) maximum number of tokens is 500, but the actual time may vary depending on the condition of Colab and the status of the Claude API.

In [ ]:
paragraph_summarizations = []

# First, we summarize each section that has been split up separately.
for index, chunk in enumerate(chunks):

    if index == 0:
        # Record the start time.
        start = time.time()

        # Construct summarization prompt.
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

        # Step1: It starts by running a prompt on a small portion of the data, generating initial output.
        first_paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # Record the result.
        paragraph_summarizations.append(first_paragraph_summarization)

        # Calculate the execution time and round it to 2 decimal places.
        cost_time = round(time.time() - start, 2)

        # Print the summary and its length.
        print(f"----------------------------Summary of Segment {index + 1}----------------------------\n")
        for text in textwrap.wrap(response, 80):
            print(f"{text}\n")
        print(f"Length of summary for segment {index + 1}: {len(response)}")
        print(f"Time taken to generate summary for segment {index + 1}: {cost_time} sec.\n")


    else:
        # Record the start time.
        start = time.time()

        # Step2: For each following document, the previous output is fed in along with the new document.
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的內容: {chunk}"""

        # Construct refinement prompt for summarization.
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_text)

        # Step3: The LLM is instructed to refine the output based on the new document's information.
        paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # Record the result.
        paragraph_summarizations.append(paragraph_summarization)

        # Calculate the execution time and round it to 2 decimal places.
        cost_time = round(time.time() - start, 2)

        # print results.
        print(f"----------------------------Summary of the First {index + 1} Segments----------------------------\n")
        for text in textwrap.wrap(paragraph_summarization, 80):
            print(f"{text}\n")
        print(f"Length of summary for the first {index + 1} segments: {len(paragraph_summarization)}")
        print(f"Time taken to generate summary for the first {index + 1} segments: {cost_time} sec.\n")

    # Step4: This process continues iteratively until all documents have been processed.

In [ ]:
''' In this block, you can modify your desired output path of final summary. '''

output_path = f"./final-summary-{suffix}-claude-refinement.txt"

# If you need to convert Simplified Chinese to Traditional Chinese, please set this option to True; otherwise, set it to False.
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # Creating an instance of OpenCC for Simplified to Traditional Chinese conversion.
    cc = OpenCC('s2t')
    paragraph_summarizations[-1] = cc.convert(paragraph_summarizations[-1])

# Output your final summary
with open(output_path, "w") as fp:
    fp.write(paragraph_summarizations[-1])

# Show the result.
print(f"Final summary has been saved to {output_path}")
print(f"\n===== Below is the final summary ({len(paragraph_summarizations[-1])} words) =====\n")
for text in textwrap.wrap(paragraph_summarizations[-1], 80):
    print(text)

# Part5 - Check the correctness of the submission file


In [ ]:
# Check the correctness of the submission file.
import json
import re

your_submission_path = "YOUR_SUBMISSION_PATH"

def check_format(your_submission_path):

    final_score = 0

    # check the extension of the file.
    if not your_submission_path.endswith(".json"):
        print("Please save your submission file in JSON format.")

    else:
        try:
            with open(your_submission_path, "r") as fp:
                your_submission = json.load(fp)

            evaluation_result = your_submission["history"][0]["messages"][1]["content"]

            if "總分：" not in evaluation_result:
                # Correct format: 總分: <你的分數>
                print("Please make sure that the correct format of final score is included in the evaluation result.")
                print("The correct format is 總分: <你的分數>. For example, 總分: 97")
                return False, final_score

            evaluation_result = evaluation_result.strip()
            score_pattern = r"總分：\d+"
            score = re.findall(score_pattern, evaluation_result)

            if score:
                final_score = score[-1].replace("總分：", "")
                if "/100" in final_score:
                    final_score = final_score.replace("/100", "")
            else:
                print("Please make sure that the final score is included in the evaluation result.")
                return False, final_score

        except:
            print("Open the file failed. Please check the file path or save your submission file in correct JSON format")
            return False, final_score

    return True, final_score

format_correctness, final_score = check_format(your_submission_path)
if format_correctness== True:
    print("The format of your submission file is correct.")
    print(f"Your final score is {final_score}.")
else:
    print("The format of your submission file is wrong.")
    print("Please check the format of your submission file.")

The format of your submission file is correct.
Your final score is 100.
